<a href="https://colab.research.google.com/github/Spnetic-5/clarifAI/blob/main/LSTMmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests

In [ ]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')


In [ ]:
# response.text

In [ ]:
data = response.text.split('\n')
data = data[253:]
data[0]

'  From fairest creatures we desire increase,'

In [ ]:
len(data)

124204

In [ ]:
data = " ".join(data)

In [ ]:
# Removing Punctuations

def clean_data(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [ w.translate(table) for w in tokens ]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens


In [ ]:
tokens = clean_data(data)
print(tokens[:50])
# len(tokens) #len(set(tokens))

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [ ]:
length = 50 + 1  #input + output length
lines = []
for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
    break
# print(len(lines)) 
 

## LSTM Model and prepare x and y


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines) 
sequences = tokenizer.texts_to_sequences(lines) # Assigning every unique word to integer Text->Integer

In [ ]:
sequences = np.array(sequences) # Integers -> Arrays
x, y = sequences[ : ,  :-1], sequences[ : ,-1] # x-> All 50 columns(Not 51) and rows & y-> All rows & Last column(51)
# x[0], y[0]

In [ ]:
vocab_size = len(tokenizer.word_index) + 1  
# print(vocab_size)
y = to_categorical(y,num_classes=vocab_size)
seq_length = x.shape[1]


## LSTM Model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))  # Embedding Layer
model.add(LSTM(100, return_sequences=True)) # 1st LSTM layer with 100 hidden layers
model.add(LSTM(100)) # 2nd LSTM layer with 100 hidden layers
model.add(Dense(100, activation='relu'))  #Dense layer with 100 neurons and relu activation
model.add(Dense(vocab_size, activation='softmax')) #Probability for each predicted word 

In [ ]:
model.summary() #State of model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            650450    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 13009)             1313909   
Total params: 2,115,259
Trainable params: 2,115,259
Non-trainable params: 0
_________________________________________________________________


In [ ]:
  # optimizers shape and mold your model into its most accurate possible form by futzing with the weights
model.compile(loss="categorical_crossentropy", optimizer= 'adam', metrics=['accuracy']) 


## Training Model

In [ ]:
model.fit(x, y, batch_size=256, epochs=100) # 500 epochs for better accuracy


Epoch 1/100
782/782 [==============================] - 388s 491ms/step - loss: 7.1975 - accuracy: 0.0283
Epoch 2/100
782/782 [==============================] - 389s 497ms/step - loss: 6.5386 - accuracy: 0.0408
Epoch 3/100
782/782 [==============================] - 386s 494ms/step - loss: 6.3471 - accuracy: 0.0570
Epoch 4/100
782/782 [==============================] - 387s 494ms/step - loss: 6.1385 - accuracy: 0.0731
Epoch 5/100
782/782 [==============================] - 386s 494ms/step - loss: 5.9720 - accuracy: 0.0840
Epoch 6/100
782/782 [==============================] - 384s 491ms/step - loss: 5.8437 - accuracy: 0.0934
Epoch 7/100
782/782 [==============================] - 385s 492ms/step - loss: 5.7371 - accuracy: 0.0989
Epoch 8/100
782/782 [==============================] - 384s 491ms/step - loss: 5.7006 - accuracy: 0.0981
Epoch 9/100
782/782 [==============================] - 391s 500ms/step - loss: 5.5642 - accuracy: 0.1059
Epoch 10/100
782/782 [==============================] -

In [ ]:
# Prediction
seed_text=lines[12343]
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = p
    ad_sequences(encoded, maxlen =text_seq_length, truncating='pre') # truncating to fix length

    y_predict = model.predict_classes(encoded) # Prediction of Index
    predicted_word = ''
    # Finding word of predicted index 
    for word, index in tokenizer.word.index.items():        
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text  + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)


In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)